In [ ]:
# # Install the dependencies

# !pip install networkx
# !pip install community
# !pip install python-louvain

In [1]:
import tweepy
import pandas as pd

# Import keys
from config import access_key, access_secret, consumer_key, consumer_secret

In [2]:
consumer_key = consumer_key
consumer_secret = consumer_secret
access_token = access_key
access_token_secret = access_secret

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

In [4]:
# Get list of users and put into csv

user_df = pd.read_csv('sandy_new.csv')

C:\Users\mokae\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# user_df = user_df.drop('Unnamed: 0', axis=1)
user_df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,417806456776167424,417806456776167424,2013-12-30 18:56:36 EST,2013-12-30,18:56:36,-500,455607722,quaaaasha,fetch .,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,417806388077686784,417806388077686784,2013-12-30 18:56:20 EST,2013-12-30,18:56:20,-500,293766434,strgto,Liz 🗡️,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,417806303289831424,417806303289831424,2013-12-30 18:55:59 EST,2013-12-30,18:55:59,-500,34920742,iembot_okx,IEMBot OKX,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,417805940801290240,417805940801290240,2013-12-30 18:54:33 EST,2013-12-30,18:54:33,-500,480642347,houston_media,Houston Media News,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,417805128771452928,417805128771452928,2013-12-30 18:51:19 EST,2013-12-30,18:51:19,-500,969318211,_ncpatriot_,= Linda🇺🇸Wray =,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
user1 = api.get_user(screen_name = 'jimdaytv')
user1.id

23855565

In [7]:
user_list = ['23855565']
follower_list = []
for user in user_list: 
    followers=[]
    try:
        for page in tweepy.Cursor(api.followers_ids,
user_id=user).pages():
            followers.extend(page)
            print(len(followers))
    except tweepy.TweepError:
            print('error')
            continue
    follower_list.append(followers)

5000
10000
15000
20000
25000
30000
35000
40000
45000
46205


In [8]:
df = pd.DataFrame(columns=['source','target']) #Empty DataFrame
df['target'] = follower_list[0] #Set the list of followers as the target column
df['source'] = 23855565 #Set user ID as the source 

In [9]:
df

,source,target
0,23855565,1019984678
1,23855565,1341745133897871361
2,23855565,1334982859644411910
3,23855565,1376005296527659009
4,23855565,1335670027719233538
...,...,...
46200,23855565,14215547
46201,23855565,12384052
46202,23855565,20007980
46203,23855565,23267939


In [10]:
import networkx as nx
G = nx.from_pandas_edgelist(df, 'source', 'target') #Turn df into graph
pos = nx.spring_layout(G) #specify layout for visual

MemoryError: Unable to allocate 15.9 GiB for an array with shape (46206, 46206) and data type float64

In [ ]:
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(10, 10))
plt.style.use('ggplot')
nodes = nx.draw_networkx_nodes(G, pos,
                               alpha=0.8)
nodes.set_edgecolor('k')
nx.draw_networkx_labels(G, pos, font_size=8)
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.2)

In [11]:
user_list = list(df['target']) #Use the list of followers we extracted in the code above i.e. my 450 followers
for userID in user_list:
    print(userID)
    followers = []
    follower_list = []

    # fetching the user
    user = api.get_user(userID)

    # fetching the followers_count
    followers_count = user.followers_count

    try:
        for page in tweepy.Cursor(api.followers_ids, user_id=userID).pages():
            followers.extend(page)
            print(len(followers))
            if followers_count >= 5000: #Only take first 5000 followers
                break
    except tweepy.TweepError:
        print("error")
        continue
    follower_list.append(followers)
    temp = pd.DataFrame(columns=['source', 'target'])
    temp['target'] = follower_list[0]
    temp['source'] = userID
    df = df.append(temp)
    df.to_csv("networkOfFollowers1.csv")

1019984678
error
1341745133897871361
9
1334982859644411910
1
1376005296527659009
3
1335670027719233538
11
1216356100519464960


Rate limit reached. Sleeping for: 649


KeyboardInterrupt: 

In [ ]:
follower_df = pd.read_csv("networkOfFollowers.csv") # Read into a df
G = nx.from_pandas_edgelist(follower_df, 'source', 'target')

In [ ]:
G.number_of_nodes() #Find the total number of nodes in this graph

In [ ]:
G_sorted = pd.DataFrame(sorted(G.degree, key=lambda x: x[1], reverse=True))
G_sorted.columns = ['nconst','degree']
G_sorted.head()

In [ ]:
# Filter the network down to a more manageable number of nodes --> k_core function of NetworkX
# k_core function filters out nodes with degree less than a given number, k

G_tmp = nx.k_core(G, 10) #Exclude nodes with degree less than 10

In [ ]:
# !pip install python-louvain

from community import community_louvain

partition = community_louvain.best_partition(G_tmp)

#Turn partition into dataframe
partition1 = pd.DataFrame([partition]).T
partition1 = partition1.reset_index()
partition1.columns = ['names','group']

In [ ]:
partition1

In [ ]:
G_sorted = pd.DataFrame(sorted(G_tmp.degree, key=lambda x: x[1], reverse=True))
G_sorted.columns = ['names','degree']
G_sorted.head()
dc = G_sorted

In [ ]:
dc

In [ ]:
combined = pd.merge(dc,partition1, how='left', left_on="names",right_on="names")

In [ ]:
combined

In [ ]:
pos = nx.spring_layout(G_tmp)
f, ax = plt.subplots(figsize=(10, 10))
plt.style.use('ggplot')
#cc = nx.betweenness_centrality(G2)
nodes = nx.draw_networkx_nodes(G_tmp, pos,
                               cmap=plt.cm.Set1,
                               node_color=combined['group'],
                               alpha=0.8)
nodes.set_edgecolor('k')
nx.draw_networkx_labels(G_tmp, pos, font_size=8)
nx.draw_networkx_edges(G_tmp, pos, width=1.0, alpha=0.2)
plt.savefig('twitterFollowers.png')

In [ ]:
# Export the node list and the edge list as csv files to be used with Gephi (open source network analysis and visualization software)

combined = combined.rename(columns={"names": "Id"}) #I've found Gephi really likes when your node column is called 'Id'
edges = nx.to_pandas_edgelist(G_tmp)
nodes = combined['Id']
edges.to_csv("edges.csv")
combined.to_csv("nodes.csv")

In [ ]:
# u = api.get_user()
# u.screen_name